In [11]:
import mysql.connector
from mysql.connector import Error
from bs4 import BeautifulSoup



In [12]:
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='127.0.0.1',
            database='kol_1017',
            user='root',
            password='123456789',
            port=3306
        )
        if connection.is_connected():
            print("Connected to MySQL database.")
            return connection
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    return None


In [40]:
def kol_radar_link(connection, kol_name, kol_link, main_category, sub_category):
    try:
        cursor = connection.cursor()

        # 插入資料
        insert_query = """
        INSERT INTO kol_radar_link (kol_name, kol_link, main_category, sub_category)
        VALUES (%s, %s, %s, %s)
        """
        cursor.execute(insert_query, (kol_name, kol_link, main_category, sub_category))
        connection.commit()
        print(f"Inserted: {kol_name}, {kol_link}")
    except Error as e:
        print(f"Error while inserting data: {e}")


In [44]:
categories = {
    # "Art+and+Entertainment": ["Interior+and+Exterior+Design", "Music","Painting","Podcast","Toy+Figure"],
    "Beauty+and+Fashion": ["Beauty+and+Fashion","Hair+Beauty","Luxury","Makeup","Medical+Beauty","Outfit","Perfume"],
    "Daily+Topics":["Daily+Topics","Holidays","Work"],
    "Education":["Educational+System","Language+Learning"],
    "Family":["Motherhood","Parenting"],
    "Finance":["Investment","Real+Estate"],
    "Food":["Beverage","Big+Eater","Dessert","Recipe"],
    "Gaming":["Computer+Game","Live+Streaming","Mobile+Game"],
    "Life+Style":["DIY"],
    "Media+Entertainment":["Dancing","Drama","Movie"],
    "Relationship":["LGBT","Marriage","Wedding"],
    "Sports":["Athletics","Baseball","Basketball","Biking","Fitting","Golf","Hiking+and+Climbing","Swimming","Tennis","Yoga"],
    "Technology":["Consumer+Electronics"],
    "Transportation":["Car","Motorcycle"],
    "Travel":["Camping","Family+Travel","Japan+Travel","Korea+Travel","Taiwan+Travel","Southeast+Asia+Travel"],
    
}

only_category = ["Climate+and+Environment","Fortunetelling","Photography","Religion","Shopping+Promoiton","Adult"]

#總共幾種類
category_count = len(categories)+len(only_category)
#總共幾個子類
sub_category_count = 0
for key in categories:
    sub_category_count += len(categories[key])
    
sub_category_count=sub_category_count+len(only_category)
print("總共有{}個類別，{}個子類".format(category_count, sub_category_count))


總共有20個類別，55個子類


In [ ]:
    # "Beauty+and+Fashion": ["Beauty+and+Fashion","Hair+Beauty","Luxury","Makeup","Medical+Beauty","Outfit","Perfume"],
    # "Daily+Topics":["Daily+Topics","Holidays","Work"],
    # "Education":["Educational+System","Language+Learning"],
    # "Family":["Motherhood","Parenting"],
    # "Finance":["Investment","Real+Estate"],
    # "Food":["Beverage","Big+Eater","Dessert","Recipe"],
    # "Gaming":["Computer+Game","Live+Streaming","Mobile+Game"],
    # "Life+Style":["DIY"],
    # "Media+Entertainment":["Dancing","Drama","Movie"],
    # "Relationship":["LGBT","Marriage","Wedding"],
    # "Sports":["Athletics","Baseball","Basketball","Biking","Fitting","Golf","Hiking+and+Climbing","Swimming","Tennis","Yoga"],
    # "Technology":["Consumer+Electronics"],
    # "Transportation":["Car","Motorcycle"],
    # "Travel":["Camping","Family+Travel","Japan+Travel","Korea+Travel","Taiwan+Travel","Southeast+Asia+Travel"],
    
}

only_category = ["Climate+and+Environment","Fortunetelling","Photography","Religion","Shopping+Promoiton","Adult"]

#總共幾種類
category_count = len(categories)+len(only_category)
#總共幾個子類
sub_category_count = 0
for key in categories:
    sub_category_count += len(categories[key])
    
sub_category_count=sub_category_count+len(only_category)
print("總共有{}個類別，{}個子類".format(category_count, sub_category_count))


In [42]:
from DrissionPage import ChromiumPage
# 瀏覽器初始化
page = ChromiumPage()
# 模擬爬取資料
def scrape_data():
    for main_category, subcategories in categories.items():
        for sub_category in subcategories:
            url = f"https://app.kolradar.com/influencers-ranking?category={main_category}%2F{sub_category}&countryCode=tw&from=sideMenu&kolBusinessType=1&platform=ig&rankType=word_of_mouth_volume"
            page.get(url)
            page.wait(5)
            print(f"Category: {main_category}/{sub_category}")

            # 爬取 KOL 資訊
            kol_lists = page.eles('.ant-table-row ant-table-row-level-0')
            for kol_list in kol_lists:
                kol_name = kol_list.ele('.kol-name-badge__KOLName-sc-99869888-1 OBdeZ').text
                cells = kol_list.eles('.ant-table-cell')
                link = cells[1].html
            
                # 使用 BeautifulSoup 解析 HTML
                soup = BeautifulSoup(link, 'html.parser')
                link_tag = soup.find('a')  # 尋找 <a> 標籤
                if link_tag and 'href' in link_tag.attrs:
                    kol_link = "https://app.kolradar.com/" + link_tag['href']
                    print(f"KOL Name: {kol_name}, Link: {kol_link}")
                    
                else:
                    print(f"KOL Name: {kol_name}, No link found")
                kol_radar_link(connection, kol_name, kol_link, main_category, sub_category)
    return kol_name, kol_link, main_category, sub_category



In [ ]:
connection = create_connection()

if connection:
    try:
        scrape_data()
    finally:
        connection.close()
        print("Database connection closed.")


Connected to MySQL database.
Category: Beauty+and+Fashion/Beauty+and+Fashion
Category: Beauty+and+Fashion/Hair+Beauty
KOL Name: My Daisy Life, Link: https://app.kolradar.com//kol/48ddcc99-5033-4ae0-9f02-5cf1ee915d45?from=socialAccountRanking
Error while inserting data: 1062 (23000): Duplicate entry 'My Daisy Life' for key 'kol_radar_link.PRIMARY'
KOL Name: Jen Wu, Link: https://app.kolradar.com//kol/3d301db9-635f-4867-998b-f8a44c7c105e?from=socialAccountRanking
Error while inserting data: 1062 (23000): Duplicate entry 'Jen Wu' for key 'kol_radar_link.PRIMARY'
KOL Name: J-Bao賤葆, Link: https://app.kolradar.com//kol/db7a0a64-963a-43b5-9e45-9dd991c2f57d?from=socialAccountRanking
Error while inserting data: 1062 (23000): Duplicate entry 'J-Bao賤葆' for key 'kol_radar_link.PRIMARY'
KOL Name: 楊祐寧, Link: https://app.kolradar.com//kol/00003215-a124-442b-970c-ff86bc97b389?from=socialAccountRanking
Error while inserting data: 1062 (23000): Duplicate entry '楊祐寧' for key 'kol_radar_link.PRIMARY'
KOL 

: 